In [ ]:
import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import urllib
import webbrowser
from bs4 import BeautifulSoup
import csv
from datetime import datetime
import time
from webbot import Browser
import json

## ETAPE 1 : SE CONNECTER A ATOM

In [ ]:
def access_atom(driver):
    """
    Accède à la plateforme Phoebus en utilisant un objet de navigateur.

    Cette fonction effectue les étapes suivantes :
    1. Navigue vers la page d'accueil de Atom Archives.
    2. Clique sur le bouton "Ouverture de session".
    3. Utilise les identifiants stockés dans le fichier 'credentials.json' pour se connecter.

    Args:
        driver: Objet de navigateur permettant d'automatiser les interactions avec la page web.
    Note:
        Assurez-vous d'avoir un fichier 'credentials.json' contenant les champs 'email' et 'password'
        pour effectuer la connexion à Phoebus.
    """
        
    driver.go_to('https://atom-archives.unil.ch/index.php')
    driver.click('Ouverture de session', tag='button')
    
    with open('credentials.json', 'r') as jsonFile:
        credentials = json.load(jsonFile)
        
    driver.type(credentials['email'], id='email')
    driver.type(credentials['password'], id='password')
    driver.click('Ouverture de session')

# LANCER LA FONCTION
driver = Browser()
access_atom(driver)

## ETAPE 2 : ACQUISITION DES DONNEES

In [ ]:
def extract_autoritePhoebus(csv_path):
    """
    Extrait les noms à partir du fichier CSV contenant les noms et les données récupérées.
    Ensuite, elle extrait
    les autorités de la colonne 'autoritesPHOEBUS' du DataFrame.

    Args:
        csv_path (str): Chemin vers le fichier CSV contenant les données récupérées.

    Returns:
        autorities (list): Liste des données récupérées.
    """
    df = pd.read_csv(csv_path, na_values=['NULL'], delimiter=',') #delimiter dépend de la configuration du separator du csv   
    # Extraire les noms de la colonne "autoritesPHOEBUS"
    autorities = df['autoritesPHOEBUS'].tolist()
    
    return autorities
    
# LANCEMENT DE LA FONCTION
csv_path = '01_autorite.csv'
autorities = extract_autoritePhoebus(csv_path)
# Afficher les noms (à titre de vérification)
print(autorities)

## ETAPE 3 : CONSTRUCTION DE L'URL DE RECHERCHE

In [ ]:
def search_autoritie (autorities_list) :
    """
    Génère des URL de recherche pour chaque nom de la liste.

    Cette fonction prend une liste de noms (autorités), extrait le nom de la première autorité
    de chaque entrée, construit une URL de recherche pour cette autorité dans Phoebus, et
    ajoute l'URL à une liste.
    La recherche avancée : NOM Prénom dans le champ Forme de nom autorisé ; Dates d'existence : vide

    Args:
        autorities_list (list): Liste des autorités à rechercher.

    Returns:
        search_urls (list): Liste d'URLs de recherche générées pour chaque autorité.
   """
    search_urls = []
    
    for autority_phoebus in autorities_list:
        autority = str(autority_phoebus)
        # Construire l'URL de recherche
        url_autority = autority.replace(' ','+')
        # recherche avancée dans Phoebus : noms dans forme de nom autorisée ;champ vide : rien
        search_url = f'https://atom-archives.unil.ch/index.php/actor/browse?sq0="{url_autority}"&sf0=authorizedFormOfName&repository=&hasDigitalObject=&entityType=&emptyField=&relatedType='

        search_urls.append(search_url)
        driver.go_to(search_url)
    return search_urls

# LANCEMENT DE LA FONCTION
search_urls = search_autoritie(autorities)
print(search_urls)

## ETAPE 4 : RECUPERER L'URL DE L'EDITION DE L'AUTORITE

In [ ]:
def browse_autorites(search_urls):
    """
    Parcourt les URL de recherche et récupère les liens vers les notices d'autorité correspondantes.

    Cette fonction prend une liste d'URL de recherche, accède à chaque URL, et récupère les liens
    vers les notices d'autorité correspondantes. Les liens sont ajoutés à une liste.

    Args:
        search_urls (list): Liste d'URLs de recherche générées pour chaque autorité.

    Returns:
        hrefs (list): Liste des liens vers les notices d'autorité.
    """
    
    hrefs_list = []
    
    for search_url in search_urls:
        hrefs =[] # Liste des liens pour une URL de recherche spécifique
        try:
            # Récupérer le contenu HTML depuis l'URL
            response = requests.get(search_url)
            response.raise_for_status()  # Vérifier si la requête a réussi
            html_content = response.text
            soup = BeautifulSoup(html_content, 'html.parser')

            # Trouver la balise <p> avec la classe "title"
            title_paragraphs = soup.find_all('p', {'class': 'title'})
            
            for title_paragraph in title_paragraphs:
                # Accéder à l'attribut 'href' de chaque balise <a> dans la balise <p>
                link_tag = title_paragraph.find('a', href=True)

                if link_tag and 'href' in link_tag.attrs:
                    href = link_tag['href']
                    hrefs.append(href)
                else:
                    print(f"Etape 4 : Erreur pour {search_url} : Balise <a> ou attribut 'href' non trouvé.")
          
            if not hrefs :
                print(f"AUCUN RESULTAT dans PHŒBUS pour {search_url}\n")
                hrefs_list.append("NULL")
            else:
                hrefs_list.append(hrefs)
                
        except requests.RequestException as e:
            print(f"Etape 4 : Erreur de requête {search_url} : {e}")   
            hrefs_list.append("ECHEC REQUETE")
         
    return hrefs_list

# LANCEMENT DE LA FONCTION
hrefs_list = browse_autorites(search_urls)
print(hrefs_list)

## ETAPE 5 : CREATION DU FICHIER CSV POUR LA GESTION DES RESULTATS MULTIPLES ET NULL

In [ ]:
def write_to_csv(hrefs_list, autorities):
    """
    Écrit les noms d'autorités avec le nbre de résultats trouvés ainsi que leurs résultats, dans un fichier CSV.

    Args:
        hrefs_list (list): Liste de listes des liens vers les notices d'autorité.
        autorities (list): Liste des autorités.
    """
    with open('02_autoritiesOnPhoebus.csv', 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
                
        # Écrire l'en-tête du CSV
        header = ['authorizedFormOfName', 'nbre de notices','notice 1', 'notice 2', 'notice 3', 'notice 4','notice 5']  # Ajoutez autant de colonnes de liens que nécessaire
        csv_writer.writerow(header)

        # Écrire les données
        for i, hrefs in enumerate(hrefs_list):
            autority_name=autorities[i]
            if hrefs == "NULL": # Si AUCUN RESULTAT alors indiquer le nbre de notice '0' et résultat NULL
                row = [autority_name, 0, f'https://atom-archives.unil.ch/index.php/botorite']
            else :
                nbr_notices = len(hrefs)  # Ajouter le nombre de notices dans la liste
                prefixed_hrefs = [f'https://atom-archives.unil.ch{href}' for href in hrefs]             
                row = [autority_name, nbr_notices, *prefixed_hrefs]
            csv_writer.writerow(row)

# LANCEMENT DE LA FONCTION
write_to_csv(hrefs_list, autorities)


## ETAPE 6 : CONTRÔLE HUMAIN 

1) contrôler les lignes indiquant 'O' (= pas de résultat trouvé), il ne devrait pas en avoir ! 
<br>
2) en cas de résultats multiples, sélectionner le bon, copier/coller dans la colonne "notice 1"

## ETAPE 7 : ACQUISITION DES URL DE RECHERCHE TRIEE

In [ ]:
def extract_hrefs_list_cleaned(csv_path_href):
    """
    Extrait les urls de Phoebus de la colonne 'url_Phoebus' du DataFrame.

    Args:
        csv_path_href (str): Chemin vers le fichier CSV contenant les données récupérées.

    Returns:
        hrefs_list_cleaned (list): Liste des données récupérées.
    """
    df = pd.read_csv(csv_path_href, na_values=['NULL'], delimiter=',') #delimiter dépend de la configuration du separator du csv   
    # Extraire les noms de la colonne "autoritesPHOEBUS"
    autorities = df['notice 1'].tolist()
    
    return autorities
    
# LANCEMENT DE LA FONCTION
csv_path_href = '02_autoritiesOnPhoebus.csv'
hrefs_list_cleaned = extract_hrefs_list_cleaned(csv_path_href)
# Afficher les noms (à titre de vérification)
print(hrefs_list_cleaned)


## ETAPE 8: CREATION DU FICHIER CSV CONTENANT UNE PARTIE DE L'URL PHOEBUS ET L'IDENTIFIANT IDREF

In [ ]:
def two_columns_csv(csv_path, hrefs_list, output_csv_path) :
    """
    Cette fonction crée un fichier CSV avec deux colonnes à partir de 01_noticesACompleter et de hrefs_list.
    Il créé le fichier csv à transmettre à IdRef pour créer un renvoi depuis idRef à Phoebus.

    Args:
        csv_path (str): Chemin du fichier CSV d'entrée.
        hrefs_list (list): Liste d'URLs à utiliser dans la première colonne du fichier de sortie.
        output_csv_path (str): Chemin du fichier CSV de sortie à créer.

    Returns:
        list: Une liste contenant les identifiants IdRef extraits du fichier CSV d'entrée.
    """
    
    # Charger le fichier CSV d'entrée
    with open(csv_path, 'r') as input_file:
        reader = csv.reader(input_file)
        rows = list(reader)

    # Extraire l'identifiant Idref du fichier CSV sans l'en-tête
    idRef= [row[1] for row in rows[1:]]   
    
    # Créer une liste de tuples avec les valeurs de hrefs_list et les identifiants IdRef
    data = list(zip(autorities, hrefs_list_cleaned, idRef))
    
     # Écrire dans le fichier CSV 03_phoebusEtIdRef
    with open(output_csv_path, 'w', newline='', encoding='utf-8') as output_file:
        writer = csv.writer(output_file, delimiter=',')
        
        # Écrire l'en-tête si nécessaire
        writer.writerow(['authorizedFormOfName','url_phoebus', 'Sources'])
        # Écrire les données
        writer.writerows(data)
        

#LANCEMENT
output_csv_path = '03_phoebusEtIdRef.csv'
two_columns_csv(csv_path, hrefs_list_cleaned, output_csv_path)

## ETAPE 9 : VERIFICATION DU NOMBRE DE LIGNES

In [ ]:
def compare_nb_lignes_csv(file1_path, file2_path, file3_path):
    """
    Compare le nombre de lignes de deux fichiers CSV.

    Args:
        file1_path (str): Chemin du fichier 01_autorite.
        file2_path (str): Chemin du fichier 02_autoritiesOnPhoebus.
        file3_path (str): Chemin du fichier 03_phoebusEtIdRef.

    Returns:
        str: Message de comparaison.
    """
    # Charger les fichiers CSV en DataFrames
    df1 = pd.read_csv(file1_path)
    df2 = pd.read_csv(file2_path)
    df3 = pd.read_csv(file3_path)

    # Obtenir le nombre de lignes de chaque DataFrame
    nb_lignes_df1 = len(df1)
    nb_lignes_df2 = len(df2)
    nb_lignes_df3 = len(df3)

    # Comparer le nombre de lignes
    if nb_lignes_df1 == nb_lignes_df2 == nb_lignes_df3 :
        message = f"BRAVO. Les fichiers csv 01_autorite, 02_autoritiesOnPhoebus et 03_phoebusEtIdRef le même nombre de lignes : {nb_lignes_df1}. \n\n{file1_path} possède {nb_lignes_df1} lignes\n{file2_path} possède {nb_lignes_df2} lignes\n{file3_path} possède {nb_lignes_df3} lignes"
    else:
        message = f"ATTENTION. LES FICHIERS 01_autorite et 02_autoritiesOnPhoebus ONT UN NOMBRE DE LIGNES DIFFERENT : {file1_path} A {nb_lignes_df1} LIGNES, TANDIS QUE {file2_path} A {nb_lignes_df2} LIGNES. \n{file3_path} possède {nb_lignes_df3} lignes.\n Contrôle et corrige !"
    
    return message

# LANCEMENT DE LA FONCTION
file1_path = '01_autorite.csv'
file2_path = '02_autoritiesOnPhoebus.csv'
file3_path = '03_phoebusEtIdRef.csv'

resultat_comparison = compare_nb_lignes_csv(file1_path, file2_path, file3_path)
print(resultat_comparison)

## LANCEMENT DES FONCTIONS

In [ ]:
# ETAPE 1
driver = Browser()
access_atom(driver)

# ETAPE 2
csv_path = '01_autorite.csv'
autorities = extract_autoritePhoebus(csv_path)

# ETAPE 3
search_urls = search_autoritie(autorities)

# ETAPE 4
hrefs_list = browse_autorites(search_urls)

# ETAPE 5
write_to_csv(hrefs_list, autorities)



## ETAPE 6 : CONTRÔLE HUMAIN 

1) contrôler les lignes indiquant 'O' (= pas de résultat trouvé), il ne devrait pas en avoir ! 
<br>
2) en cas de résultats multiples, sélectionner le bon, copier/coller dans la colonne "notice 1"

## LANCEMENT DES FONCTIONS

In [ ]:
# ETAPE 7
csv_path_href = '02_autoritiesOnPhoebus.csv'
hrefs_list_cleaned = extract_hrefs_list_cleaned(csv_path_href)

# ETAPE 8
output_csv_path = '03_phoebusEtIdRef.csv'
two_columns_csv(csv_path, hrefs_list_cleaned, output_csv_path)

# ETAPE 9
file1_path = '01_autorite.csv'
file2_path = '02_autoritiesOnPhoebus.csv'
file3_path = '03_phoebusEtIdRef.csv'

resultat_comparison = compare_nb_lignes_csv(file1_path, file2_path, file3_path)